In [17]:
import fastf1
from fastf1.ergast import Ergast

In [18]:
import requests
import pandas as pd

def get_driver_standings():
    url = f"http://ergast.com/api/f1/{2023}/driverStandings.json"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        standings_data = data['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']

        flattened_data = []

        for standing in standings_data:
            driver_data = standing.get('Driver', {})
            constructors = standing.get('Constructors', [])
            constructor_data = constructors[0] if constructors else {}

            flattened_item = {
                'position': standing.get('position'),
                'positionText': standing.get('positionText'),
                'points': standing.get('points'),
                'wins': standing.get('wins'),
                'driverId': driver_data.get('driverId'),
                'driverPermanentNumber': driver_data.get('permanentNumber'),
                'driverCode': driver_data.get('code'),
                'driverName': driver_data.get('givenName', '') + ' ' + driver_data.get('familyName', ''),
                'driverDOB': driver_data.get('dateOfBirth'),
                'driverNationality': driver_data.get('nationality'),
                'constructorId': constructor_data.get('constructorId'),
                'constructorName': constructor_data.get('name'),
                'constructorNationality': constructor_data.get('nationality')
            }
            flattened_data.append(flattened_item)

        df = pd.DataFrame(flattened_data)
        return df
    else:
        print(f"HTTP request failed with status {response.status_code}.")
        return None



In [19]:
def calculate_max_points_for_remaining_season():
    POINTS_FOR_SPRINT = 8 + 25 + 1  # Winning the sprint, race and fastest lap
    POINTS_FOR_CONVENTIONAL = 25 + 1  # Winning the race and fastest lap

    events = fastf1.events.get_events_remaining()
    # Count how many sprints and conventional races are left
    sprint_events = len(events.loc[events["EventFormat"] == "sprint"])
    conventional_events = len(events.loc[events["EventFormat"] == "conventional"])

    # Calculate points for each
    sprint_points = sprint_events * POINTS_FOR_SPRINT
    conventional_points = conventional_events * POINTS_FOR_CONVENTIONAL

    return sprint_points + conventional_points

In [22]:
def calculate_who_can_win(driver_standings, max_points):
    LEADER_POINTS = int(driver_standings.loc[0]['points'])

    for i, _ in enumerate(driver_standings.iterrows()):
        driver = driver_standings.loc[i]
        driver_max_points = int(driver["points"]) + max_points
        can_win = 'No' if driver_max_points < LEADER_POINTS else 'Yes'

        print(f"{driver['position']}: {driver['driverName']}, "
              f"Current points: {driver['points']}, "
              f"Theoretical max points: {driver_max_points}, "
              f"Can win: {can_win}")

In [23]:
# Get the current drivers standings
driver_standings = get_driver_standings()

# Get the maximum amount of points
points = calculate_max_points_for_remaining_season()

# Print which drivers can still win
calculate_who_can_win(driver_standings, points)

1: Max Verstappen, Current points: 144, Theoretical max points: 430, Can win: Yes
2: Sergio Pérez, Current points: 105, Theoretical max points: 391, Can win: Yes
3: Fernando Alonso, Current points: 93, Theoretical max points: 379, Can win: Yes
4: Lewis Hamilton, Current points: 69, Theoretical max points: 355, Can win: Yes
5: George Russell, Current points: 50, Theoretical max points: 336, Can win: Yes
6: Carlos Sainz, Current points: 48, Theoretical max points: 334, Can win: Yes
7: Charles Leclerc, Current points: 42, Theoretical max points: 328, Can win: Yes
8: Lance Stroll, Current points: 27, Theoretical max points: 313, Can win: Yes
9: Esteban Ocon, Current points: 21, Theoretical max points: 307, Can win: Yes
10: Pierre Gasly, Current points: 14, Theoretical max points: 300, Can win: Yes
11: Lando Norris, Current points: 12, Theoretical max points: 298, Can win: Yes
12: Nico Hülkenberg, Current points: 6, Theoretical max points: 292, Can win: Yes
13: Oscar Piastri, Current points